In [12]:
import numpy as np
import pandas as pd
import geopandas as gpd
from pyscipopt import Model, quicksum, multidict

In [14]:
# for I, d make a dictionary of planning units to number of students
pu_data = gpd.read_file('GIS_files/pu_with_proj.geojson').set_index('pu_2324_84') # from git folder
pu_data = pu_data['final_proj'].to_dict()

I, o = multidict(pu_data)

In [32]:
# read in planning unit and school geospatial files
schools = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/dps_hs_locations.geojson') # from git folder
schools = schools.to_crs('EPSG:4326')
pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/geospatial files/pu_shape.geojson') # can't remember
pu = pu.to_crs('EPSG:4326')

# add planning unit number to school data
for i, geometry in enumerate(pu['geometry']):
    in_geometry = geometry.contains(schools['geometry'])
    pu_id = pu.loc[i, 'OBJECTID']

    schools.loc[in_geometry, 'pu'] = pu_id

In [70]:
school_cap = pd.read_csv('/Users/leahwallihan/Durham_school_planning/CSV files/dps_capacities_20250328_toshare.csv') # from school capacities of google drive folder

existing_sites = []
potential_sites = I.copy()

# add capacities to dataframe
schools['capacity'] = 0
for i, school in schools.iterrows():
    pu_id = int(school['pu'])
    existing_sites.append(pu_id)
    if pu_id in potential_sites:
        potential_sites.remove(pu_id)

    # try matching by the first word of school name
    base_name = school['name'].split(' ')[0]
    match = school_cap[school_cap['name'].str.contains(base_name, na=False)]

    if not match.empty:
        cap = match['sch_cap_hb90'].values[0]
        schools.loc[i, 'capacity'] = cap

In [72]:
schools

,OBJECTID,facilityid,sch_id6,sch_id,name,address,phone,agencyurl,operhours,spectype,...,calendar_2324,calendar_2425,choice_2324,choice_2425,choice_2526,x_ncplane,y_ncplane,geometry,pu,capacity
0,11,27253,320368,368,Southern High School,"800 Clayton Road Durham, NC 27703",919-560-3968,http://southern.dpsnc.net/pages/Southern_High,9:15am - 4:15pm,Magnet School,...,Traditional,Traditional,Energy & Sustainability,Energy & Sustainability,Energy & Sustainability,2050552.39,818903.23,POINT (-78.8291 35.9999),602.0,1540
1,15,87951,320325,325,Hillside High School,3727 Fayetteville Street Durham NC 27707,919-560-3925,http://hillside.dpsnc.net/pages/Hillside_High,9:15am - 4:15pm,None,...,Traditional,Traditional,International Baccalaureate,International Baccalaureate,International Baccalaureate,2028531.50,801909.91,POINT (-78.9036 35.9533),290.0,1535
2,56,6494,320356,356,Northern High School,4622 N Roxboro Road Durham NC 27712,919-560-3956,http://northern.dpsnc.net/pages/Northern_High,9:15am - 4:15pm,None,...,Traditional,Traditional,None,None,None,2029463.13,841220.63,POINT (-78.90032 36.06129),45.0,1600
3,57,12065,320365,365,Riverside High School,"3218 Rose of Sharon Road Durham, NC 27712",919-560-3965,http://www.edlinesites.net/pages/Riverside_High,9:15am - 4:15pm,None,...,Traditional,Traditional,None,None,None,2018531.26,843582.30,POINT (-78.9373 36.0678),566.0,1540
4,58,76194,320312,312,Jordan High School,6806 Garrett Road Durham NC 27707,919-560-3912,http://jordan.dpsnc.net/pages/Jordan_High,9:15am - 4:15pm,None,...,Traditional,Traditional,None,None,None,2011193.93,790759.00,POINT (-78.9622 35.9227),507.0,1810


In [ ]:


alpha = 1.0
beta = -1.0

T = {}

for i in I:
    denom = sum(w[j]**alpha * d[i, j]**beta for j in J)
    for j in J:
        numerator = w[j]**alpha * d[i, j]**beta
        T[i, j] = O[i] * numerator / denom if denom != 0 else 0